In [ ]:
import os
from scripts_2.utilities import merge_audio_inside_folder, get_frames_per_interval
from googletrans import Translator

%store -r WIDTH
%store -r HEIGHT
%store -r TEXT_IMAGE_GEN  
%store -r SAVE_FOLDER
%store -r PROJECT_NAME
%store -r SHOW_OUTPUT
%store -r IMG_NUMBER
%store -r EXCLUDE_IMG_GEN
%store -r PROMPT_IMG
%store -r TEXT_VOICE_GEN
%store -r NEGATIVE_P
%store -r MODEL_14
%store -r RECORD_VIDEO_AFTER
%store -r FPS
%store -r LANGUAGES

# from scripts_2.utilities import 
""" final_final_deforum, audio_duration, srt_interval =
get_frames_per_interval(project_folder, 0.5, 4) #root  project_folder, delay and cadence"""

project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"
translator = Translator()



In [ ]:
def create_srt(text, initial_time, final_time, indx, filename):
  initial_mil = int(round(initial_time % 1,3)*1000)
  final_mil = int(round(final_time % 1,3)*1000)
  initial_time = int(initial_time)
  final_time = int(final_time)

  minutes, seconds = divmod(initial_time, 60)
  hours, minutes = divmod(minutes, 60)
  initial_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}'

  minutes, seconds = divmod(final_time, 60)
  hours, minutes = divmod(minutes, 60)
  final_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}'


  with open(f"{filename}//en.srt" , 'w', encoding='utf-8') as f:
    f.write(f'{int(indx)+1}\n')
    f.write(f'{initial_time} --> {final_time}\n')
    f.write(f'{text}\n')
    f.write('\n')
  
  for lang in LANGUAGES:
    transl = translator.translate(text, src='en', dest=lang).text
    with open(f"{filename}//{lang}.srt" , 'w', encoding='utf-8') as f:
      f.write(f'{int(indx)+1}\n')
      f.write(f'{initial_time} --> {final_time}\n')
      f.write(f'{transl}\n')
      f.write('\n')

def merge_srt_files(filenames, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i, filename in enumerate(filenames):
            with open(filename, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)
                if i < len(filenames) - 1:
                    outfile.write('\n')

In [ ]:
full_folder = f'{project_folder}/full'
audio_folder = f'{project_folder}/audio'
srt_folder = f"{project_folder}/srt"

#get durations
_, audio_duration, srt_interval = get_frames_per_interval(project_folder, 0.25, 4)
if not os.path.exists(full_folder):
    os.makedirs(full_folder)

if (not os.path.exists(srt_folder)):
    os.makedirs(srt_folder)

if os.path.exists(f'{full_folder}/full.wav'):
    os.remove(f'{full_folder}/full.wav')

# merge_audio_inside_folder(audio_folder,full_folder,audio_duration)
merge_audio_inside_folder(audio_folder,full_folder)

for index in srt_interval:
    current_str_folder = f'{srt_folder}/{index:07d}'
    if (not os.path.exists(current_str_folder)):
        os.makedirs(current_str_folder)
    create_srt(TEXT_VOICE_GEN[str(index)], srt_interval[index]['start_time'], srt_interval[index]['end_time'], index,current_str_folder)

filenames = []    
filenames_other = {}
for lan in LANGUAGES:
    filenames_other[lan] = []
#merge srt
for folder in os.listdir(f"{project_folder}//srt"):
        for filename in os.listdir(f"{project_folder}//srt//{folder}"):
            if (filename[-6:] == 'en.srt'):                    
                filenames.append(f"{project_folder}//srt//{folder}//{filename}")
            elif(filename[-3:] == 'srt'):
                
                lan = filename[-6:-4]
            
                filenames_other[lan] = filenames_other[lan] + [f"{project_folder}//srt//{folder}//{filename}"]
    


if len(filenames) > 0:
    merge_srt_files(filenames, f"{project_folder}//full//full_hd.srt")
    for lang_mod in filenames_other:
        merge_srt_files(filenames_other[lang_mod], f"{project_folder}//full//{lang_mod}.srt")





""" merge_audio_inside_folder(audio_folder,audio_folder) """


